In [5]:
!pip install evaluate -q

In [6]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from evaluate import load
import numpy as np
import random
import torch

In [7]:
# Clean notebook metadata
try:
    from ipython_mallorca.nbclean import clean_notebook_metadata
    clean_notebook_metadata()
    print("✅ Cleaned notebook metadata — you can now safely upload to GitHub!")
except ImportError:
    print("Skipping metadata cleaning: ipython_mallorca not found.")
except Exception as e:
    print(f"Error cleaning metadata: {e}")

Skipping metadata cleaning: ipython_mallorca not found.


In [8]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [9]:
dataset = load_dataset("noob123/imdb_review_3000")

print(dataset)

if "train" in dataset:
    full = dataset["train"]
else:
    full = dataset[list(dataset.keys())[0]]

text_col = "review"
label_col = "sentiment"

def encode_labels(example):
    example["label"] = 1 if example[label_col].lower() == "positive" else 0
    return example

seed = random.randint(0, 1000)
print(f"🔹 Random seed: {seed}")
set_seed(seed)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


imdb_review.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2999 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 2999
    })
})
🔹 Random seed: 202


In [10]:
split1 = full.train_test_split(test_size=0.2, seed=seed)
train_part = split1["train"]
rest = split1["test"]
split2 = rest.train_test_split(test_size=0.5, seed=seed)
val_part = split2["train"]
test_part = split2["test"]

train_part = train_part.map(encode_labels)
val_part = val_part.map(encode_labels)
test_part = test_part.map(encode_labels)

print(f"Train: {len(train_part)}, Val: {len(val_part)}, Test: {len(test_part)}")

Map:   0%|          | 0/2399 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Train: 2399, Val: 300, Test: 300


In [11]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_fn(examples):
    tokenized = tokenizer(examples[text_col], padding="max_length", truncation=True, max_length=64)
    tokenized["label"] = examples["label"]
    return tokenized

train_dataset = train_part.map(tokenize_fn, batched=True)
val_dataset = val_part.map(tokenize_fn, batched=True)
test_dataset = test_part.map(tokenize_fn, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/2399 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [12]:
metric = load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels, average="weighted")


In [13]:
def run_experiment(name, lr, epochs):
    print(f"\n🚀 Running {name}: lr={lr}, epochs={epochs}")
    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

    args = TrainingArguments(
        output_dir=f"./results/{name}",
        eval_strategy="epoch", # Changed from evaluation_strategy
        save_strategy="epoch",
        learning_rate=lr,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=epochs,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        logging_dir="./logs",
        logging_steps=50,
        fp16=torch.cuda.is_available(),  # ✅ Use mixed precision if GPU available
        report_to="none",  # Disable extra logging
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    res = trainer.evaluate()
    print(f"✅ Validation F1: {res['eval_f1']:.4f}")
    return trainer, res

In [14]:
experiments = [
    ("fast_run_1", 5e-5, 2),
    ("fast_run_2", 3e-5, 3),
    ("fast_run_3", 2e-5, 3)
]

results = []
for name, lr, ep in experiments:
    tr, res = run_experiment(name, lr, ep)
    results.append((name, tr, res))


🚀 Running fast_run_1: lr=5e-05, epochs=2


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,F1
1,0.504700,0.412805,0.799733
2,0.331900,0.410693,0.809791


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


...

✅ Validation F1: 0.8099

🚀 Running fast_run_3: lr=2e-05, epochs=3


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,F1
1,0.511400,0.442606,0.795762
2,0.378200,0.428232,0.796443
3,0.276400,0.446132,0.806667


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


✅ Validation F1: 0.8067
